In [37]:
import pandas as pd
from collections import Counter
from sklearn.mixture import GaussianMixture
import numpy as np

In [3]:
%pwd

'/opt/notebooks/CS 6220/Assignments'

## Spambase

In [9]:
spambase = pd.read_csv('data/spambase/spambase.data', header=None)

In [20]:
Y_train = spambase[57]
X_train = spambase[[col for col in spambase.columns if col !=57]]

In [23]:
estimator = GaussianMixture(n_components=10, covariance_type='full')

In [25]:
estimator.fit(X_train)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [29]:
yhat_train = estimator.predict(X_train)

In [31]:
counter = Counter() 
for i in yhat_train: counter[i] += 1

In [32]:
counter.most_common()

[(3, 1784),
 (0, 1053),
 (9, 952),
 (8, 623),
 (1, 82),
 (7, 54),
 (4, 48),
 (2, 3),
 (6, 1),
 (5, 1)]

In [35]:
purity_df = pd.DataFrame([yhat_train, Y_train]).transpose()
purity_df.columns = ['yhat_test', 'y_test']

purity_numerator = 0
purity_denominator = len(purity_df)

for cluster in purity_df['yhat_test'].unique():
    
    purity_numerator += int(purity_df[purity_df['yhat_test'] == cluster].groupby('y_test')\
                                .count()\
                                .sort_values('yhat_test', ascending=False)\
                                .iloc[0])
purity_numerator/purity_denominator

0.7237557052814606

In [43]:
gini_coeffs = np.zeros([len(purity_df['yhat_test'].unique()), 2])
i = 0

for cluster in yhat_train:

    if (i < len(set(list(yhat_train)))):
        
        gini_coeffs[i][0] = sum(((purity_df[purity_df['yhat_test'] == cluster].groupby('y_test')\
                                    .count()\
                                    .sort_values('yhat_test', ascending=False)\
                            /len(purity_df[purity_df['yhat_test'] == cluster]))\
                            **2)\
                            ['yhat_test'])

        gini_coeffs[i][1] = len(purity_df[purity_df['yhat_test'] == cluster])

        i += 1
    
    else: break

sum(gini_coeffs[:,0] * gini_coeffs[:,1]) / sum(gini_coeffs[:,1])

0.52538025687703716